In [50]:
import pandas as pd
import glob
import numpy as np
import os

In [2]:
files = glob.glob('stimuli/*.png')

In [4]:
len(files)

31

In [10]:
stimuli_df = pd.DataFrame({'target1':files[:10], 'target2':files[10:20], 'distracter':files[20:30]})
stimuli_df.to_csv('stimuli.csv')

In [11]:
stimuli_df

,target1,target2,distracter
0,stimuli\Baracke.png,stimuli\Fitness_Chocolate.png,stimuli\Parachocolate.PNG
1,stimuli\Branflakes.png,stimuli\FuzeTea.png,stimuli\Prince.PNG
2,stimuli\Branflakesextra.png,stimuli\FuzeTeaGreen.png,stimuli\Quaker.PNG
3,stimuli\Cafeelite.png,stimuli\GelDeLin.png,stimuli\Sano99.PNG
4,stimuli\Chocolatemilk.png,stimuli\HEINZ.PNG,stimuli\Smarties.PNG
5,stimuli\ChocolateSpread.png,stimuli\KifKef.PNG,stimuli\Splendid.PNG
6,stimuli\CocaCola2L.png,stimuli\Kikkoman.PNG,stimuli\Tapuchips.PNG
7,stimuli\Colgate.png,stimuli\Milka.PNG,stimuli\Tuna_ToGo.png
8,stimuli\Filtuna.png,stimuli\Neviot.PNG,stimuli\Wissotzky.png
9,stimuli\Fitness.png,stimuli\Oreo.PNG,stimuli\YadMordechai.png


In [5]:
pd.DataFrame({'image':files}).to_csv('stimuli.csv')

In [276]:
block_data_files = glob.glob('data/neuro_DN_001_1_block*.csv')
experiment_data = pd.DataFrame([])
for block_file in block_data_files:
    block_data = pd.read_csv(block_file, index_col=0)
    experiment_data = pd.concat([experiment_data, block_data])
# remove missed trials
# experiment_data = experiment_data.loc[experiment_data.rt!='None']
# average responses over blocks
avg_product_choices = experiment_data[['image_path', 'choice']].groupby('image_path').mean()
# sort products from highest (0) to lowset (-1)
avg_product_choices = avg_product_choices.sort_values(by='choice', ascending=False)
avg_product_choices = avg_product_choices.reset_index()
# choose target products, the first 10 
targets = avg_product_choices.iloc[:10]

In [277]:
# define the differences between targets that we want to choose
diffs = [9, 7, 6, 5, 3, 1]
# get indices corresponding to each kind of diff from the targets, i.e., 1 and 10 for diff==9, 3 and 9 for diff==6, etc.
target_indices = []
for diff in diffs:
    target_diff_indices = []
    for i in range(len(targets)):
        if i + diff < len(targets):
            diff_array = np.array([i, i + diff, diff])
            target_diff_indices.append(diff_array)
    target_indices = target_indices + target_diff_indices

# choose one randomly diff index for each diff, minimizing the amount of products that will be selected more than once
target_indices = np.array(target_indices)
target_indices_df = pd.DataFrame(target_indices, columns=['target1', 'target2', 'diff'])
selected_indices = {diff: np.array([]) for diff in diffs}
selected_products = set()
product_counts = {product: 0 for product in range(len(targets))}

for diff in diffs:
    # get indices with the current diff
    diff_indices = target_indices_df[target_indices_df['diff'] == diff].copy()
    # remove selected products from the diff indices
    least_selected_products = diff_indices[~diff_indices['target1'].isin(selected_products) & ~diff_indices['target2'].isin(selected_products)]
    # if all products have been selected at least once, choose the least selected pair
    if least_selected_products.empty:
        diff_indices.loc[:, 'target1_count'] = diff_indices['target1'].apply(lambda x: product_counts[x])
        diff_indices.loc[:, 'target2_count'] = diff_indices['target2'].apply(lambda x: product_counts[x])
        diff_indices.loc[:, 'pair_count'] = diff_indices['target1_count'] + diff_indices['target2_count']
        # get the pairs with minimal pair count
        minimal_count = diff_indices['pair_count'].min()
        least_selected_products = diff_indices[diff_indices['pair_count'] == minimal_count]
    # ranodmlly choose one of the least selected pairs
    least_selected_products = least_selected_products.sample()
    # save targets indices
    target1, target2 = least_selected_products['target1'].values[0], least_selected_products['target2'].values[0]
    selected_indices[diff] = np.array([target1, target2])
    # save selected products, increase count
    selected_products.add(target1)
    selected_products.add(target2)
    product_counts[target1] += 1
    product_counts[target2] += 1

In [278]:
target_index_pair_df = pd.DataFrame(selected_indices, index=['target1', 'target2']).T

In [279]:
distractors = avg_product_choices.iloc[10::4]

In [280]:
distractors

,image_path,choice
10,stimuli\Kikkoman.png,64.042369
14,stimuli\KifKef.PNG,57.011742
18,stimuli\ChocolateSpread.png,44.069677
22,stimuli\Zippersmallbag.png,29.349242
26,stimuli\Quakermaybe.png,20.930571
30,stimuli\Tuna_ToGo.png,5.407757


In [289]:
# construct the final stimuli dataframe
fmri_stimuli_df = pd.DataFrame([], columns=['target1', 'target2', 'distractor', 
                                            'target1_rank', 'target2_rank', 'distractor_rank', 
                                            'target1_choice', 'target2_choice', 'distractor_choice'])
for diff in target_index_pair_df.index:
    target1_index, target2_index = target_index_pair_df.loc[diff]
    target1 = targets.loc[target1_index, 'image_path']
    target2 = targets.loc[target2_index, 'image_path']
    target1_choice = targets.loc[target1_index, 'choice']
    target2_choice = targets.loc[target2_index, 'choice']
    for distractor_i in distractors.index:
        distractor = distractors.loc[distractor_i, 'image_path']
        distractor_choice = distractors.loc[distractor_i, 'choice']
        stimuli_triad = pd.DataFrame({  'target1': target1, 'target2': target2, 'distractor': distractor, 
                                        'target1_rank': target1_index, 'target2_rank': target2_index, 'distractor_rank': distractor_i,
                                        'target1_choice': target1_choice, 'target2_choice': target2_choice, 'distractor_choice': distractor_choice,
                                        'targets_diff': diff}, index=[distractor_i])
        fmri_stimuli_df = pd.concat([fmri_stimuli_df, stimuli_triad], ignore_index=True)

In [290]:
fmri_stimuli_df 

,target1,target2,distractor,target1_rank,target2_rank,distractor_rank,target1_choice,target2_choice,distractor_choice,targets_diff
0,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\Kikkoman.png,0,9,10,75.246225,66.328630,64.042369,9.0
1,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\KifKef.PNG,0,9,14,75.246225,66.328630,57.011742,9.0
2,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\ChocolateSpread.png,0,9,18,75.246225,66.328630,44.069677,9.0
3,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\Zippersmallbag.png,0,9,22,75.246225,66.328630,29.349242,9.0
4,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\Quakermaybe.png,0,9,26,75.246225,66.328630,20.930571,9.0
5,stimuli\Heinz.png,stimuli\Filtunamaybe.png,stimuli\Tuna_ToGo.png,0,9,30,75.246225,66.328630,5.407757,9.0
6,stimuli\Fitness_Chocolate.png,stimuli\Barackemaybe.png,stimuli\Kikkoman.png,1,8,10,73.288385,68.261163,64.042369,7.0
7,stimuli\Fitness_Chocolate.png,stimuli\Barackemaybe.png,stimuli\KifKef.PNG,1,8,14,73.288385,68.261163,57.011742,7.0
8,stimuli\Fitness_Chocolate.png,stimuli\Barackemaybe.png,stimuli\ChocolateSpread.png,1,8,18,73.288385,68.261163,44.069677,7.0
9,stimuli\Fitness_Chocolate.png,stimuli\Barackemaybe.png,stimuli\Zippersmallbag.png,1,8,22,73.288385,68.261163,29.349242,7.0


In [ ]:
analyzed_data = pd.DataFrame({'image':trials.data['image_path'], 'choice':trials.data['choice']})
# sort product by choice
### TO DO:
# average the choices over blocks
analyzed_data.sort_values(by='choice', ascending=False, inplace=True)
# take top 10 to be targets
### TO DO:
# Diffs: 9 (like 1 vs 10), 7 (1 vs 8, 2 vs 9, etc. choose random), 6, 5, 3, 1
# avoid choosing the same product twice
targets_1 = analyzed_data.iloc[:10:2]
targets_2 = analyzed_data.iloc[1:10:2]
# take next 20 to be distractors
# choose 6 out of 20 distractors, with equal jumps. say jump of 3
distractors = analyzed_data.iloc[10:20:2]
stimuli_df = pd.DataFrame({'target1':targets_1.image_path.values, 'target1_rating':targets_1.choice.values,
                        'target2':targets_2.image_path.values, 'target2_rating':targets_2.choice.values,
                        'distracter':distractors.image_path.values, 'distracter_rating':distractors.choice.values})